In [1]:
import sys
from PyQt4.QtCore import pyqtSlot
from PyQt4 import QtGui, QtCore
import glob, openpyxl, pandas as pd, numpy as np
from datetime import datetime, timedelta
from PyQt4.QtGui import QMessageBox

class Window(QtGui.QMainWindow):

    def __init__(self):
        
        super(Window, self).__init__()
        self.setGeometry(50, 50, 500, 700)
        self.setWindowTitle("RMR Calculator")

        extractAction = QtGui.QAction("Quit", self)
        extractAction.setShortcut("Ctrl+Q")
        extractAction.setStatusTip('Quit The App')
        extractAction.triggered.connect(self.close)

        self.statusBar()

        mainMenu = self.menuBar()
        
        fileMenu = mainMenu.addMenu('&File')
        fileMenu.addAction(extractAction)

        self.home()

    def home(self):
        
        # Create textbox
        self.textbox = QtGui.QLineEdit(self)
        self.textbox.move(130,80)
        self.textbox.resize(300, 20)
        
        # Create a button in the window
        self.btn = QtGui.QPushButton("Select File", self)
        self.btn.clicked.connect(self.select_file)
        self.btn.move(20,80)
        self.btn.resize(100,20)
        
        self.btn = QtGui.QPushButton("Load Data", self)
        filename = self.textbox.text()
        self.btn.clicked.connect(self.load_data)
        self.btn.move(320, 120)
        self.btn.resize(100,20)
        
        self.show()

    def select_file(self):
        fileName = QtGui.QFileDialog.getOpenFileName(self, 'Select File','*.xlsx')
        self.textbox.setText(fileName)
    
    def load_data(self):
        
        def create_dataframe():
            columns = ['Patient ID', 'Age', 'Sex','Height (in)', 'Height (cm)', 'Weight (Ib)',
           'Weight (kg)', 'Time (min)', 'VO2/kg STPD (ml/kg/m)', 'VO2 STPD (L/min)',
          '(ml/min)', 'VCO2 STPD (L/min)', '(ml/min)', 'VE STPD (L/min)', 'RQ', 
           'RR (BPM)', 'VE/VO2 (BT/ST)', 'METS', 'TM SPD(mph)', 'Calculate Start Row',
            'Calcualte End Row','Calcualted RMR']
            df = pd.DataFrame(columns = columns)
            return df
        
        def get_Patient_Info(sheet,data):
            #print("start get patient info")
            # Create a list to save patient's information
            item = []
    
            # Read cell values in row 7 to the list
            for col in range(1,8):
                value = sheet.cell(row = 7,column = col).value
                if value is not None:
                    if is_number(value):
                        item.append(value)
                    if (value == 'F' or value == 'M'):
                        item.append(value)
                    if (value == 'Male'):
                        item.append('M')
                    if (value == 'Female'):
                        item.append('F')
          
            # Read cell values in row 8 to the list
            for col in range(1,13):
                value = sheet.cell(row = 8,column = col).value
                if value is not None:
                        if is_number(value):
                            item.append(value)
   
            # Create a new list to save patient information 
            data = append_Multi_Item(data,item)
            #print(data)
    
            #print("end get patient info")
            return data
        
        ## Function to append multiple item in a item list 
        def append_Multi_Item(data, itemList):
            #print("start append_multi_item")
            for i in range(0,(len(itemList))):
                data.append(itemList[i])
                #print("end append_multi_item")
            return data
        
        ## Function to check the value is numeric or not
        def is_number(s):
            #print("start is_number")
            try:
                float(s)
                #print("end is_number")
                return True
            except ValueError:
                #print("end is_number")
                return False
        
        def get_start_of_data(sheet):
    
            #print("start get_start_of_data")
            # Save the index of the number of start line of raw data
            for row in range(27,32):
                value = sheet.cell(row = row, column = 1).value
                if value == 'min  ':
                    data_start = row + 2
                    break
            #print("end get_start_of_data")
            return data_start
        
        def get_end_of_data(sheet):
    
            print("start get_end_of_data")
            # Get max line
            max_row = sheet.max_row
    
    
            # Save the index of the numebr of end line of raw data
            try: 
                #print(max_row)
                cell_5 = sheet.cell(row = max_row, column = 5)
                #cell_6 = sheet.cell(row = max_row, column = 6)
                if (cell_5.value is not None):
                    #print(cell_6.value)
                    #printnt(cell_5.value)
                    if (is_number(cell_5.value)):
                        data_end = max_row
                    else:
                        try:
                            cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                            if (cell_5__rowminusone.value is not None): 
                                #print(cell_5__rowminusone.value)
                                if (is_number(cell_5__rowminusone.value)):
                                    data_end = max_row - 1
                        except IndexError:
                            #print("....indexError")
                            exit = False
                            for row in range(max_row - 80 , max_row-1):
                                #print(row)
                                try: 
                                    cell = sheet.cell(row = row, column = 2)
                                    if (cell.value is None):
                                        #print("isnone")
                                        #print(row)
                                        if (is_data_end(sheet, row-1)):
                                            #print(row - 1)
                                            data_end = row - 1
                                            exit = True
                                        if (exit):
                                            break
                                except IndexError:
                                    #print("little indexerror")
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                else: 
                    #print("else")
                    exit = False
                    try:
                        #rint("try/...")
                        cell_5__rowminusone = sheet.cell(row = max_row - 1, column = 5)
                        #print("cell5")
                        if (cell_5__rowminusone.value is not None): 
                            #print(cell_5__rowminusone.value)
                            if (is_number(cell_5__rowminusone.value)):
                                data_end = max_row - 1
                        else:
                            for row in range(max_row - 80 , max_row-1):
                                #print(row)
                                try: 
                                    cell = sheet.cell(row = row, column = 2)
                                    if (cell.value is None):
                                        #print("isnone")
                                        #print(row)
                                        if (is_data_end(sheet, row-1)):
                                            #print(row - 1)
                                            data_end = row - 1
                                            exit = True
                                        if (exit):
                                            break
                                except IndexError:
                                    #print("little indexerror")
                                    data_end = row - 1
                                    exit = True
                                if (exit):
                                    break
                    except IndexError:
                        #print("indexError")
                        exit = False
                        for row in range(max_row - 80 , max_row):
                            #print(row)
                            try: 
                                cell = sheet.cell(row = row, column = 1)
                                if (cell.value is None):
                                    #print("isnone")
                                    #print(row)
                                    if (is_data_end(sheet, row-1)):
                                        #print(row - 1)
                                        data_end = row - 1
                                        exit = True
                                    if (exit):
                                        break
                            except IndexError:
                                #print("little indexerror")
                                data_end = row - 1
                                exit = True
                            if (exit):
                                break
            except IndexError: 
                #print("big indexerror")
                exit = False
                for row in range(max_row-25, max_row):
                    try:
                        #print(row)
                        cell = sheet.cell(row = row, column = 1)
                    except IndexError: 
                        data_end = row - 1
                        exit = True
                    if (exit):
                        break
            print("end get_end_of_data")
            return data_end
        
        def get_data(sheet, data, yellow, df, columns):
    
            #print("start get_data")
            # Get the start line of raw data
            data_start = get_start_of_data(sheet)
    
            # Get the end line of raw data
            data_end = get_end_of_data(sheet)
    
            # Read data in excel by row and append the data to data list and df
            for row in range(data_start, data_end + 1):
                data_copy = []
                data_copy = append_Multi_Item(data_copy, data)
                #print(data_copy)
                #print(row)
                data_excel = get_data_in_row(sheet, row)
                data_copy = append_Multi_Item(data_copy, data_excel)
                #print(data_copy)
                #print(data)
                data_copy = append_Multi_Item(data_copy, yellow)
                #print(data_copy)
                #print(data)
                df2 = pd.DataFrame([data_copy], columns = columns)
                df = df.append(df2)
    
            #print("end get_data")
            return df
        
        def get_data_in_column(sheet, data_start, data_end, col_number):
    
            #print("start get_data_in_column")
            data = []
            #Get and save the data in column 
            for row in range(data_start, data_end + 1):
                value = sheet.cell(row = row, column = col_number).value
                data.append(value)
            #print("end get_data_in_column")
            return data
        
        def get_data_in_row(sheet, row_number):
    
            #print("start get_data_in_row")
            data = []
            #Get and save the data in row
            for col in range(1, 13):
                value = sheet.cell(row = row_number, column = col).value
                if col == 4 or col == 6:
                    value = data[col-2] * 1000
                data.append(value)
            #print("end get_data_in_row")
            return data
        
        def is_data_end(sheet, row):
            #print("start is_data_end")
            cell_9 = sheet.cell(row = row, column = 9)
            cell_10 = sheet.cell(row = row, column = 10)
            if (is_number(cell_9.value) and is_number(cell_10.value)):
                #print("end is_data_end")
                return True
            else:
                #print("end is_data_end")
                return False
        
        filename = self.textbox.text()
        if filename:
            df = create_dataframe()
            
             # open excel work book and sheet1
            wb = openpyxl.load_workbook(filename, use_iterators= True, data_only = True)
            sheet_1 = wb.get_sheet_by_name(name = 'Sheet1')
            #sheet_2 = wb.get_sheet_by_name(name = 'Subject Report')
    
            # Call function to update data list with age, sex, height and weight
            data = get_Patient_Info(sheet_1, data)
    
            # Call function to get the list of start and end index of yellow region
            yellow = get_yellow_index(sheet_1)
    
            # Call function to get the data and save the data to dataframe
            df = get_data(sheet_1, data, yellow, df, columns)
            print(df)
    
        else: 
            print("else")
            msg = QtGui.QMessageBox()
            msg.setInformativeText("You have to select the data file first in order to calculate.")
            msg.setWindowTitle("Warning")
            msg.setStandardButtons(QMessageBox.Ok)
            msg.buttonClicked.connect(msg.close)
            ret = msg.exec_()
    

def run():
    app = QtGui.QApplication(sys.argv)
    GUI = Window()
    sys.exit(app.exec_())


run()     

ImportError: No module named 'PyQt4'